In [ ]:
#frames = FrameHandler(video_path)
#results = frames.video_process(skip_rate, frame_no)


#VideoHandler()
#    - Check if sequence
#    - Return list of ranges

#video handler two main methods:
    #routes video path (folder path) to frame handler called process_video
    #check resulting list to find a sequence 

#FrameHandler()
#    - Check if it is cheating or not
#    - Type of cheating
#    - timestamp

#directory = os.listdir(path)
#for file in directory:
#    video =  VideoHandler(file)
    #frames =  FrameHandler(file)
#    results = video.video_process(skip_rate, frame_no)

In [21]:
import cv2
import mediapipe as mp
import datetime
import numpy as np
import pandas as pd
import time
import os
import shutil
import math

In [22]:
!pip install --upgrade pandas

In [23]:
import copy
from datetime import datetime

In [32]:
def make_timestamp(mill_sec):
    
        total_sec = (mill_sec / 1000)%60
        seconds = total_sec % (24 * 3600)
        hours = str(int(seconds // 3600)).zfill(2)
        minutes = str(int((mill_sec/1000) // 60)).zfill(2)
        
        
        
        sec = total_sec % 60
        int_sec = int(sec)
        
        msec = str(int((sec-int_sec)*100)).zfill(2)
        int_sec2 = str(int_sec).zfill(2)
        
        
        min_sec = f"{hours}:{minutes}:{int_sec2},{msec}" 

        return min_sec
    
def download_csv(p_tuple):
        new_tp = p_tuple[:]
        for tup in new_tp:
            x = tup[1]
            tup[1] = make_timestamp(x)
        df = pd.DataFrame(new_tp, columns=['Label','Timestamp', 'Pose'])
        
        #df['Timestamp'] = df['Timestamp'].apply(lambda x: x.strftime('%hh:%mm:%ss'))
        df.to_csv('output.csv', index=False) #date_format = '%hh:%mm:%ss,%f',   encoding='utf-8-sig'

        #[suspicious, timestamps, pose]
def get_subtitle(p_tuple):
        new_tp = p_tuple[:]
        prev = None
        end = None
        sub_list = []
        for tup in new_tp:
            if prev is not None:
                x = tup[1]
                news = x #make_timestamp(x)
                if tup[2] == pose_prev:
                    end = news
                else:
                    if end is not None:
                        prev_tp = make_timestamp(prev)
                        end_tp  = make_timestamp(end)
                        if end - prev > 2000:
                            sub_list.append([prev_tp, end_tp, pose_prev])
                    prev = None
                    end = None
            else:
                pose_prev = tup[2]
                prev = tup[1]
        i = 0
        
        string = ''
        for  n in sub_list:
            i+=1
            x = f"{i}" + "\n" + f"{n[0]}" + " --> " + f"{n[1]}" + "\n" + f"{n[2]}" + "\n\n"
            string = string + x
        return string

In [33]:
class FrameHandler:
    
    """ Summary of FrameHandler.
    This class handles all the frames of the given video.
    """
    
    #frame_no = 0
    def __init__(self): 
        
        """
        Attributes:
        
        face_mesh             gets the mesh of the given frame
        
        left_eye_landmarks    all landmark coordinates of left eye in MediaPipe
        
        right_eye_landmarks    all landmark coordinates of right eye in MediaPipe
        
        """
        self.face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=10, min_detection_confidence=0.5, min_tracking_confidence=0.5)    
        
        self.left_eye_landmarks =  [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163,7]
        self.right_eye_landmarks = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]

        
    
    def __draw_mask(self, ph, pw, eye_region, gray):
        
        """
        Drawing mask for eyes.
        
        Parameters
        ----------
        ph : 
            height of the frame
        pw : 
             width of the frame
        eye region:
            array of an eye's coordinates
        gray: 
            gray scale of the frame
 
        Returns
        -------
        masked version of the eye
        
        """
        
        mask = np.zeros((ph, pw), np.uint8)
        cv2.polylines(mask, [eye_region], True, 255,2)
        cv2.fillPoly(mask,[eye_region],255)
        eye = cv2.bitwise_and(gray, gray, mask=mask)   
        
        return eye
    
    def __draw_lines(self, p_frame, point1, point2, point3): #frame,nose, reye, leye, mbrow
        """not used"""
    
        frames = cv2.line(p_frame, (int(point1[0]), int(point1[1]) ), (int(point2[0]), int(point2[1]) ), (255,0,0), 1)
        frames = cv2.line(frames,  (int(point1[0]), int(point1[1]) ), (int(point3[0]), int(point3[1]) ), (255,0,0), 1)
        frames = cv2.line(frames,  (int(point2[0]), int(point2[1]) ), (int(point3[0]), int(point3[1]) ), (255,0,0), 1)

        return frames

    def __find_euc_distance(self, point1,point2):
        """
        Finding euclidean distance between 2 points.
        
        Parameters
        ----------
        point1 : 
            coordinates of the start point
        point2: 
            coordinates of the end point
 
        Returns
        -------
        float:
            distance between point1 and point2
        """
        distance = math.sqrt(((point1[0] - point2[0]) ** 2) + ((point1[1] - point2[1]) ** 2)) 
        return distance

    def __area_heron(self, s1, s2, s3):
        """
        Finding area of the triangle with their given sides s1,s2,s3.
        
        Parameters
        ----------
        s1 : 
            length of s1
        s2: 
            length of s2
        s3:
            length of s3
 
        Returns
        -------
        float:
            area of the triangle
        """
        s = (s1+ s2 + s3) / 2
        area = math.sqrt(s * (s - s3) * (s - s2) * (s - s1))
        return area

    def __find_area_tri(self, point1,point2,point3):
        """
        Finding area of a triangle with 3 points.
        
        Parameters
        ----------
        point1 : 
            coordinates of the point1
        point2: 
            coordinates of the point2
        point3:
            coordinates of the point3
        Returns
        -------
        float:
            area of the triangle
        """
        dist_p12p2 =  self.__find_euc_distance(point1, point2) #nose 2 mbrow

        dist_p12p3 =  self.__find_euc_distance(point1, point3) #nose 2 eye

        dist_p22p3 =  self.__find_euc_distance(point2, point3) #mbrow 2 eye

        area = self.__area_heron(dist_p12p2, dist_p12p3,dist_p22p3)

        return area

    def __draw_max_min(self, eye_region):
        
        min_x, max_x  = np.min(eye_region[:,0]),  np.max(eye_region[:,0])
        min_y, max_y  = np.min(eye_region[:,1]), np.max(eye_region[:,1])
        
        return min_x, max_x, min_y, max_y
        
    def __check_face(self, p_mesh): #returns face count
        face_count = 0
        
        if p_mesh.multi_face_landmarks:
            face_count = len(p_mesh.multi_face_landmarks)
            
        return face_count   
        
    def __check_head_pose(self, p_mesh, p_frame, p_check=False, p_vgaze=None, p_hgaze=None):
        
        """
        Check if head movements happen in the frame.
        
        Parameters
        ----------
        p_mesh : 
            mesh of face in the frame
        p_frame: 
            given_frame
        p_check:
            True - method is called for calibration, False - method is called anywhere else
        p_vgaze:
            calibration value for vertical head movement (up, down)
        p_hgaze:
            calibration value for horizontal head movement (left, right)
            
        Returns
        -------
        Boolean:
            True - if head movement happens, False - not
        """
        
        hmax_thres, hmin_thres = 0.55, 0.45
        vmax_thres, vmin_thres = 0.48, 0.4

        head_pose = False
        ph, pw, pc = p_frame.shape

        rmost_eye, lmost_eye, nose_tip = 263, 33, 1
        lside, rside, mchin            = 93, 323, 200
        mbrow, forehead, lnose_side    = 9, 10, 48
        rnose_side, chin               = 278, 152
        

        for face_landmarks in p_mesh.multi_face_landmarks:

            for idx, lm in enumerate(face_landmarks.landmark):

                if idx == nose_tip:              
                    nose_2d = (lm.x * pw, lm.y * ph)  
                    nose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == rmost_eye: #rightmost eye point 
                    reye_2d = (lm.x * pw, lm.y * ph)  
                    reye_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == lmost_eye: #leftmost eye point
                    leye_2d = (lm.x * pw, lm.y * ph)  
                    leye_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == mbrow: #middle of brows
                    mbrow_2d = (lm.x * pw, lm.y * ph)  
                    mbrow_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == lnose_side: #leftmost point of the nose
                    lnose_2d = (lm.x * pw, lm.y * ph)  
                    lnose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == rnose_side: #rightmost point of the nose
                    rnose_2d = (lm.x * pw, lm.y * ph)  
                    rnose_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)

                elif idx == mchin: #middle of the chin point
                    mchin_2d = (lm.x * pw, lm.y * ph)  
                    mchin_3d = (lm.x * pw, lm.y * ph, lm.z * 3000)          

        area_left  = self.__find_area_tri(nose_2d, mbrow_2d, leye_2d) #left eye nose brow
        area_right = self.__find_area_tri(nose_2d, mbrow_2d, reye_2d) #right ey nose brow

        area_bottom = self.__find_area_tri(mchin_2d, lnose_2d, rnose_2d) #chin to 
        area_top    = self.__find_area_tri(mbrow_2d, lnose_2d, rnose_2d) # forehead to 


        horizontal_ratio = area_left / (area_left+area_right) #for left right movement

        vertical_ratio   = area_bottom / (area_bottom+area_top) #for up down movement


        #pcheck indicates whether or not method will calculate calibration values 
        
        if p_check:  #method is called to calculate calibration values
            
            return horizontal_ratio, vertical_ratio
        
        else: #method is called for head pose detecetion
            
            if p_vgaze is not None or p_hgaze is not None: # head pose detection with calibration values
                
                if horizontal_ratio < p_hgaze * 0.7 or horizontal_ratio > p_hgaze * 1.3: 
                        head_pose = True
                    
                if not head_pose:
                    if vertical_ratio < p_vgaze * 0.87 or vertical_ratio > p_vgaze * 1.15:
                        head_pose = True
                        
            else: #head pose detection without calibration values
                
                if horizontal_ratio > hmax_thres or horizontal_ratio < hmin_thres:
                    head_pose = True
                    
                if not head_pose:
                    if vertical_ratio > vmax_thres or vertical_ratio < vmin_thres:
                        head_pose = True

            return head_pose    
    
    def __check_gaze(self, p_mesh, p_frame, p_check=False, p_center_value=None): 
        
        """
        Check if eye movements happen in the frame.
        
        Parameters
        ----------
        p_mesh : 
            mesh of face in the frame
        p_frame: 
            given_frame
        p_check:
            True - method is called for calibration, False - method is called anywhere else
        p_center_value:
            calibration value when gaze is directed to the center
        
        Returns
        -------
        Boolean:
            True - if eye movement happens, False - not
        """
    
        check_ratio = False
        eps = 1e-10

        
        ph, pw, pc = p_frame.shape 
        gray = cv2.cvtColor(p_frame, cv2.COLOR_BGR2GRAY)

        left_eye_temp = []
        right_eye_temp = []

        for face_landmarks in p_mesh.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in self.left_eye_landmarks:
                    left_eye_temp.append((int(lm.x*pw),int(lm.y*ph)))
                elif idx in self.right_eye_landmarks:
                    right_eye_temp.append((int(lm.x*pw),int(lm.y*ph)))

        right_eye_region = np.array(right_eye_temp, np.int32)   
        left_eye_region  = np.array(left_eye_temp, np.int32) 

        eye_left  = self.__draw_mask(ph, pw, left_eye_region,  gray)
        eye_right = self.__draw_mask(ph, pw, right_eye_region, gray)
 
        min_xl, max_xl, min_yl, max_yl =  self.__draw_max_min(left_eye_region)
        min_xr, max_xr, min_yr, max_yr =  self.__draw_max_min(right_eye_region)

        gray_eye_left  = eye_left[min_yl:max_yl, min_xl:max_xl]
        gray_eye_right = eye_right[min_yr:max_yr, min_xr:max_xr]

        eye_left  = cv2.resize(gray_eye_left,  None, fx=5, fy=5)
        eye_right = cv2.resize(gray_eye_right, None, fx=5, fy=5)

        _, threshold_eye_left = cv2.threshold(gray_eye_left, 70, 255, cv2.THRESH_BINARY)
        _, threshold_eye_right = cv2.threshold(gray_eye_right, 70, 255, cv2.THRESH_BINARY)

        height_l, width_l = threshold_eye_left.shape
        height_r, width_r = threshold_eye_right.shape

        left_left_side_threshold = threshold_eye_left[0:height_l, 0:int(width_l/2)]
        left_left_side_white = cv2.countNonZero(left_left_side_threshold)

        left_right_side_threshold = threshold_eye_left[0:height_l, int(width_l/2):width_l]
        left_right_side_white = cv2.countNonZero(left_right_side_threshold)


        right_left_side_threshold = threshold_eye_right[0:height_r, 0:int(width_r/2)]
        right_left_side_white = cv2.countNonZero(right_left_side_threshold)
        
        right_right_side_threshold = threshold_eye_right[0:height_r, int(width_r/2):width_r]
        right_right_side_white = cv2.countNonZero(right_right_side_threshold)

        if left_left_side_white == 0:
            gaze_ratio_left = 0
        elif left_right_side_white ==0:
            gaze_ratio_left = 1
        else:
            gaze_ratio_left = left_left_side_white/(left_right_side_white+left_left_side_white)

        if right_left_side_white == 0:
            gaze_ratio_right = 0
        elif right_right_side_white ==0:
            gaze_ratio_right = 1
        else:
            gaze_ratio_right = right_left_side_white/(right_right_side_white+right_left_side_white)

        gaze_ratio = (gaze_ratio_left+gaze_ratio_right)/2

        if p_check:
            return gaze_ratio
        else:
            if p_center_value is not None:
                if gaze_ratio < p_center_value * 0.7 or gaze_ratio > p_center_value * 1.3: 
                    check_ratio = True
            else:
                if gaze_ratio < 0.45 or gaze_ratio > 0.58: # <0.45 right >0.58 left
                    check_ratio =  True
                    
        return check_ratio
        
    def __calibrate(self, p_calibration_frame):
        """
        Check if head movements happen in the frame.
        
        Parameters
        ----------
        p_calibration_frame: 
            The frame which is going to used for calibrated
            
        Returns
        -------
        array:
            [
            value when gaze is directed to the center
            value when vertical head movement
            value when horizontal head movement
            ]
        """
        calibration_check = True
        calibration_frame = cv2.cvtColor(p_calibration_frame, cv2.COLOR_BGR2RGB)
        
        calibration_frame.flags.writeable = False
        
        res_mesh = self.face_mesh.process(calibration_frame)
        #res_pose = self.face_pose.process(calibration_frame)
        
        calibration_frame.flags.writeable = True
        center_values = []
        
        if res_mesh.multi_face_landmarks:
            center_gaze = self.__check_gaze(res_mesh, calibration_frame, True)
            center_head_horizontal, center_head_vertical = self.__check_head_pose(res_mesh, calibration_frame, True)
            center_values = [center_gaze, center_head_vertical, center_head_horizontal]  
        return center_values
    
    def __find_timestamp(self, p_timestamp):
        mill_sec = p_timestamp
        total_sec = mill_sec / 1000
        minutes = int(total_sec // 60)
        sec = total_sec % 60
        int_sec = int(sec)
        msec = int((sec-int_sec)*100)
        min_sec = f"{minutes}:{int_sec}:{msec}"       
        return min_sec
    
    def __calculate_motion(self,p_prev_frame, p_curr_frame, p_threshold=200):
        prev_gray = cv2.cvtColor(p_prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(p_curr_frame, cv2.COLOR_BGR2GRAY)
    
        # Compute the absolute difference between the two frames
        diff = cv2.absdiff(prev_gray, curr_gray)
    
        # Apply a binary threshold to get a binary image
        _, diff_thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    
        # Sum the non-zero pixels in the thresholded difference image
        motion_amount = cv2.countNonZero(diff_thresh)
    
        # Return True if the motion amount is above the threshold, False otherwise
        return motion_amount > p_threshold, motion_amount
        
    def process_video(self, p_videox, p_prev_frame, p_frame_no, p_calibration_path=None): #this method allow
        
        suspicious_frames = []
        frame_no = p_frame_no
        values = None
        skip = 0
        suspicious = False
        pose = 'normal'
        
        fps = p_videox.get(cv2.CAP_PROP_FPS)
        
        if p_calibration_path is not None:
            calibration_frame = cv2.imread(p_calibration_path)
            values = self.__calibrate(calibration_frame) #0 - center gaze, 1 - vertical ratio, 2 - horizontal ratio
        
        while p_videox.isOpened():  
            
            success, curr_frame = p_videox.read()
             
            if not success:
                break
            frame_no +=1
            
            
            if skip > 0:
                skip = skip - 1            
                continue
            
            timestamps = p_videox.get(cv2.CAP_PROP_POS_MSEC)
            
            #min_sec = self.__find_timestamp(timestamps)
            
            if suspicious:
                calc = True
            else:
                calc = self.__calculate_motion(p_prev_frame, curr_frame, 200)[0]
                

            if calc:
                #self.__calculate_motion(prev_frame, curr_frame, 200)[0] and suspicious == False: #(frame_no % skip_rate == 0):
                skip = 5 
                frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
                
                frame.flags.writeable = False
                
                res_mesh = self.face_mesh.process(frame)
                #res_pose = self.face_pose.process(frame)
                
                frame.flags.writeable = True
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                
                fh, fw, fc = frame.shape
                num_face =self.__check_face(res_mesh)          
    
                if num_face == 0:
                    suspicious = True
                    pose='No face detected'
                elif num_face > 1:
                    suspicious = True
                    pose = 'More than one face detected'
                else: #if num_face == 1:
                    
                    if values is not None:
                        head_pose_detected = self.__check_head_pose(res_mesh, frame, False, values[1], values[2])
                    else:
                        head_pose_detected = self.__check_head_pose(res_mesh, frame,False)                 
                    
                    if head_pose_detected:
                        suspicious = True 
                        pose = 'Head pose detected'
                    else:
                        if values is not None:
                            gaze_detected = self.__check_gaze(res_mesh, frame, False, values[0])
                        else:
                            gaze_detected = self.__check_gaze(res_mesh, frame, False)    
                            
                        if gaze_detected:
                            suspicious = True 
                            pose = 'Gaze detected'
                        else:
                            suspicious = False
                            pose = 'Normal frame'
                            
                    
                suspicious_frames.append([suspicious, timestamps, pose])#,frame_no))

            prev_frame = curr_frame.copy()
            
                            
            #if cv2.waitKey(1) & 0xFF == 27:
            #    break
        #video.release()
        #cv2.destroyAllWindows()
        
        return suspicious_frames
    
    
                    
                

In [39]:
class VideoHandler():
    
    def __init__(self,video_path):
        self.video_path = video_path
        #self.skip_rate = skip_rate
        
    def process_video(self, calibration_path=None):
        suspicious_frames = []
        frame_no = 0
        
        videox = cv2.VideoCapture(self.video_path)
        ret, prev_frame = videox.read()
        
        if not ret:
            videox.release()
            exit()
        
            
        frameObj = FrameHandler()
        
        results = frameObj.process_video(videox, prev_frame, frame_no, calibration_path)
        
        return results
    def __make_timestamp(self, mill_sec):
        total_sec = mill_sec / 1000
        minutes = str(int(total_sec // 60)).zfill(2)
        sec = total_sec % 60
        int_sec = int(sec)
        msec = str(int((sec-int_sec)*100)).zfill(2)
        int_sec = str(int_sec).zfill(2)
        min_sec = f"{minutes}:{int_sec}:{msec}" 
        
        return min_sec
    
    def __make_timestamp_msec(self,mill_sec):
      
        total_sec = (mill_sec / 1000)%60
        seconds = total_sec % (24 * 3600)
        hours = str(int(seconds // 3600)).zfill(2)
        minutes = str(int((mill_sec/1000) // 60)).zfill(2)
        
        
        
        sec = total_sec % 60
        int_sec = int(sec)
        
        msec = str(int((sec-int_sec)*100)).zfill(2)
        int_sec2 = str(int_sec).zfill(2)
        
        
        min_sec = f"{hours}:{minutes}:{int_sec2},{msec}" 

        return min_sec
    
    def check_sequence(self, presult):
        p_results =  copy.deepcopy(presult)

        sequence = [] 
        whole_sequence = []
        i = 0
        for item in p_results:  
            i+=1
            if item[0] == True:     
                sequence.append((item[1],item[2]))
                if len(sequence) == len(p_results): # and (sequence[-1][0]-sequence[0][0]) > p_mseconds:
                    whole_sequence.append(sequence)
                elif i == len(p_results) and len(sequence)>5: #and (sequence[-1][0]-sequence[0][0]) > p_mseconds: #len(sequence) > 5:
                    whole_sequence.append(sequence)
            else:  
                if  len(sequence) > 5: # and (sequence[-1][0]-sequence[0][0]):
                    whole_sequence.append(sequence)
                sequence = []
        i = 0
        whole_seq = []
        prev = None
        for seq in whole_sequence:
            i += 1
            begin = seq[0][0]
            end   = seq [-1][0]
            curr  = begin
            if prev is not None:
                if (curr - prev) < 3000:
                    s2 = seq[-1][0]
                else:
                    together = self.__make_timestamp(b1) + " - " + self.__make_timestamp(s2) + " " + str(seq[-1][1])
                    b1 = begin
                    s2 = end
                    whole_seq.append(together)                 
                if i == len(whole_sequence):
                    together = self.__make_timestamp(b1) + " - " + self.__make_timestamp(s2) + " " + str(seq[-1][1])
                    whole_seq.append(together)                
            else:
                #keep_b1 = begin
                #b1 = keep_b1
                b1 = begin
                s2 = end
        
            prev = end
            curr_begin = begin

        return whole_seq
    
    def get_subtitle(self, p_tuple):
        i = 0
        new_tp = copy.deepcopy(p_tuple)
        prev = None
        end = None
        sub_list = []
        for tup in new_tp:
            i+=1
            if prev is not None:
                news = tup[1]
                if tup[2] == pose_prev and i != len(new_tp):
                    end = news
                else:
                    if end is not None:
                        if end - prev > 1500 or  pose_prev == 'Normal frame':
                            prev_tp = self.__make_timestamp_msec(prev)
                            end_tp  = self.__make_timestamp_msec(end)
                            sub_list.append([prev_tp, end_tp, pose_prev])
                    prev = tup[1]
                    pose_prev = tup[2]
                    end = None
            else:
                pose_prev = tup[2]
                prev = tup[1]
        
        i = 0
        
        string = ''
        for  n in sub_list:
            i+=1
            x = f"{i}" + "\n" + f"{n[0]}" + " --> " + f"{n[1]}" + "\n" + f"{n[2]}" + "\n\n"
            string = string + x 
        file_address =  "\\".join(video_path.split('\\')[0:-1])+"\\subtitle_for_" + video_path.split('\\')[-1].split('.')[0]+".srt"
        f = open(file_address, "w")
        f.write(string)
        f.close()
    def download_csv(self, p_tuple):
        new_tp = copy.deepcopy(p_tuple)
        for tup in new_tp:
            x = tup[1]
            tup[1] = self.__make_timestamp_msec(x)
        df = pd.DataFrame(new_tp, columns=['Suspicious Frame?','Timestamp', 'Pose'])
        file_address =  "\\".join(video_path.split('\\')[0:-1])+"\\detailed_" + video_path.split('\\')[-1].split('.')[0]+".csv"
        df.to_csv(file_address, index=False, encoding='utf-8-sig') #date_format = '%hh:%mm:%ss,%f',   

In [40]:
path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\TestVideos\\"
folder_path = os.listdir(path)
for folder in folder_path:
    file_path = os.listdir(path+folder)
    for file in file_path:
        if file.endswith('.mp4'):
            video_path = path+folder+"\\"+file
        elif file.endswith('.jpg'):
            calibration_path = path+folder+"\\"+file
    
    video = VideoHandler(video_path)
    results = video.process_video(calibration_path)
    #new_res = copy.deepcopy(results)
    video.get_subtitle(results)
    video.download_csv(results)
    res = video.check_sequence(results)

    for line in res:
        print(line)
    
    
    

In [6]:
#skip_rate = 1
#frame_no = 0 
#directory = os.listdir(path)
#for file in directory:
#    video =  VideoHandler(file)
    #frames =  FrameHandler(file)
#    results = video.video_process(skip_rate, frame_no)

#video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\testvideo12.mp4"
#folder_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\TestVideos"
#fp = os.fsencode(folder_path)

video_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\TestVideos\\Video 22052024\\testvideo_1.mp4"
calibration_path = "C:\\Users\\99455\\Desktop\\DataScience\\Proctoring Task\\TestVideos\\Video 22052024\\calibration_photo.jpg"
#video_path =  'C:/Users/99455/Desktop/DataScience/OEP database/subject2/Jourabloo1.avi'

video = VideoHandler(video_path)
#frames = FrameHandler()
results = video.process_video(calibration_path)



    
#print(results)

[0.4686364784695664, 0.4355719977675883, 0.5029779652395643]
0.4657724287916805
0.4552440226901651
0.4566855348682529
0.4529348980725068
0.45310417612561266
0.45728010848718986
0.45412819649272307
0.45565264449737003
0.45357168560425104
0.4534092564375983
0.4537124437749827
0.45510654799922057
0.4504028479542989
0.4541203348145024
0.45223267493631003
0.4526597522108786
0.45292140908376755
0.45445708086114034
0.45355381166324044
0.4478196559431169
0.4492956308492471
0.4557038816160427
0.4522560919407489
0.4539695846921638
0.45306049658560804
0.4537009993789227
0.4542017383243461
0.45395123790636016
0.4531067810546042
0.45246779037092294
0.4557797090546187
0.45317650128037773
0.4529843168129249
0.4542208591101517
0.45536352932775037
0.4540683475921115
0.45265953547279014
0.4555638385916039
0.45384621150687526
0.4555315264825884
0.4556799456073537
0.4550814045158428
0.45479690166776177
0.4539735578752782
0.45152512298488773
0.45344018368074107
0.4526737218284012
0.45354550270132127
0.4525

0.4590912923929146
0.4624779915701749
0.4611932563442682
0.4596574060299672
0.46138274990965544
0.4613418990475789
0.46020848951119914
0.46274545343561835
0.4619957093075188
0.4614257658425451
0.4632443510648374
0.46333081049075386
0.4636717873619628
0.46076436013440386
0.46125209505729586
0.4624115248219045
0.46161857337169077
0.4622495105038292
0.4634533444386812
0.46222595929891547
0.4634180666280815
0.4597620738051502
0.4619286246655837
0.4615330685472459
0.46191310778067074
0.46260427251489994
0.46223993004751385
0.4608557885626515
0.45964667838739
0.46113311604232216
0.46455920318639005
0.4609555133065602
0.4614081938655118
0.46184022079337356
0.45897484745424977
0.4668112653456593
0.48004288826419456
0.4982943438134419
0.47829467882900434
0.4714031244060803
0.4619734732434114
0.45477317557046426
0.4594120736199827
0.4686954141346854
0.45869658570707844
0.45687423208547157
0.45168682176791203
0.45499166661551593
0.45733156919748313
0.46481540156815804
0.45559695004186024
0.459842